In [1]:
#[class , x, y, w, h , conf]
#['0', '0.0880208', '0.140741', '0.176042', '0.281481', '0.705591']

In [21]:
import os

In [22]:
#YOLO형태에서  VOC 형식으로 변환
def convertToAbsoluteValues(box):
    #box = [class , x, y, w, h , conf]
    
    xIn = ( float(box[1]) - float(box[3]/2)) # x-(w/2)
    yIn = ( float(box[2]) - float(box[4]/2)) 
    xEnd = xIn + (float(box[3]) )
    yEnd = yIn + (float(box[4]) )
    
    if xIn < 0:
        xIn = 0
    if yIn < 0:
        yIn = 0
    if xEnd >= 1:
        xEnd = 1 - 1
    if yEnd >= 1:
        yEnd = 1 - 1
       
    return [xIn, yIn, xEnd, yEnd]

In [23]:
#label과 detection 결과를 불러와 리스트에 저장
def boundingBoxes(labelPath, imagePath):
    
    detections, groundtruths, classes = [], [], []
    
    for boxtype in os.listdir(labelPath):

        boxtypeDir = os.path.join(labelPath,boxtype)

        for labelfile in os.listdir(boxtypeDir):
            filename = os.path.splitext(labelfile)[0]
            with open(os.path.join(boxtypeDir, labelfile)) as f:
                labelinfos = f.readlines()

            imgfilepath = os.path.join(imagePath, filename + ".jpg")
            img = cv.imread(imgfilepath)
            h, w, _ = img.shape

            for labelinfo in labelinfos:
                label, conf, rx1, ry1, rx2, ry2 = map(float, labelinfo.strip().split())
                x1, y1, x2, y2 = convertToAbsoluteValues((w, h), (rx1, ry1, rx2, ry2))
                boxinfo = [filename, label, conf, (x1, y1, x2, y2)]
                
                if label not in classes:
                    classes.append(label)
                
                if boxtype == "detection":
                    detections.append(boxinfo)
                else:
                    groundtruths.append(boxinfo)
                    
    classes = sorted(classes)
                
    return detections, groundtruths, classes

In [24]:
#box = [ x1, y1, x2, y2]
def getArea(box):
    """
    box의 절대값 좌표를 받아 box의 넓이를 반환
    
    """
    return (box[2] - box[0] + 1) * (box[3] - box[1] + 1)


def getUnionAreas(boxA, boxB, interArea=None):
    """
    2개의 box의 절대값 좌표를 인자로 받아 2개의 box의 넓이에서 겹치는 역역을 제외하여 합집합 영역을 반환
    """
    area_A = getArea(boxA)
    area_B = getArea(boxB)
    
    if interArea is None:
        interArea = getIntersectionArea(boxA, boxB)
        
    return float(area_A + area_B - interArea)

def getIntersectionArea(boxA, boxB):
    """
    2개의 box의 절대값 좌표를 인자로 받아서 두 box가 겹치는 영역을 반환함
    
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # intersection area
    return (xB - xA + 1) * (yB - yA + 1)

In [25]:
def boxesIntersect(boxA, boxB):
    """
    
    2개의 box의 절대값 좌표를 받아 좌표값을 비교하여 box가 곂치는지 여부를 boolean타입으로 반환
    
    
    """
    if boxA[0] > boxB[2]:
        return False  # boxA is right of boxB
    if boxB[0] > boxA[2]:
        return False  # boxA is left of boxB
    if boxA[3] < boxB[1]:
        return False  # boxA is above boxB
    if boxA[1] > boxB[3]:
        return False  # boxA is below boxB
    return True

def iou(boxA, boxB):
    """
    2개의 box의 좌표를 받아서 IoU를 반환
    """
    if boxesIntersect(boxA, boxB) is False:
        return 0
    interArea = getIntersectionArea(boxA, boxB)
    union = getUnionAreas(boxA, boxB, interArea=interArea)
    
    # intersection over union
    result = interArea / union
    assert result >= 0
    return result

In [26]:
def calculateAveragePrecision(rec, prec):
    
    mrec = [0] + [e for e in rec] + [1]
    mpre = [0] + [e for e in prec] + [0]

    for i in range(len(mpre)-1, 0, -1):
        mpre[i-1] = max(mpre[i-1], mpre[i])

    ii = []

    for i in range(len(mrec)-1):
        if mrec[1:][i] != mrec[0:-1][i]:
            ii.append(i+1)

    ap = 0
    for i in ii:
        ap = ap + np.sum((mrec[i] - mrec[i-1]) * mpre[i])
    
    return [ap, mpre[0:len(mpre)-1], mrec[0:len(mpre)-1], ii]

In [27]:
def ElevenPointInterpolatedAP(rec, prec):

    mrec = [e for e in rec]
    mpre = [e for e in prec]

    recallValues = np.linspace(0, 1, 11)
    recallValues = list(recallValues[::-1])
    rhoInterp, recallValid = [], []

    for r in recallValues:
        argGreaterRecalls = np.argwhere(mrec[:] >= r)
        pmax = 0

        if argGreaterRecalls.size != 0:
            pmax = max(mpre[argGreaterRecalls.min():])

        recallValid.append(r)
        rhoInterp.append(pmax)

    ap = sum(rhoInterp) / 11

    return [ap, rhoInterp, recallValues, None]

In [28]:
def AP(detections, groundtruths, classes, IOUThreshold = 0.3, method = 'AP'):
    
    result = []
    
    for c in classes:

        dects = [d for d in detections if d[1] == c]
        gts = [g for g in groundtruths if g[1] == c]

        npos = len(gts)

        dects = sorted(dects, key = lambda conf : conf[2], reverse=True)

        TP = np.zeros(len(dects))
        FP = np.zeros(len(dects))

        det = Counter(cc[0] for cc in gts)

        # 각 이미지별 ground truth box의 수
        # {99 : 2, 380 : 4, ....}
        # {99 : [0, 0], 380 : [0, 0, 0, 0], ...}
        for key, val in det.items():
            det[key] = np.zeros(val)


        for d in range(len(dects)):


            gt = [gt for gt in gts if gt[0] == dects[d][0]]

            iouMax = 0

            for j in range(len(gt)):
                iou1 = iou(dects[d][3], gt[j][3])
                if iou1 > iouMax:
                    iouMax = iou1
                    jmax = j

            if iouMax >= IOUThreshold:
                if det[dects[d][0]][jmax] == 0:
                    TP[d] = 1
                    det[dects[d][0]][jmax] = 1
                else:
                    FP[d] = 1
            else:
                FP[d] = 1

        acc_FP = np.cumsum(FP)
        acc_TP = np.cumsum(TP)
        rec = acc_TP / npos
        prec = np.divide(acc_TP, (acc_FP + acc_TP))

        if method == "AP":
            [ap, mpre, mrec, ii] = calculateAveragePrecision(rec, prec)
        else:
            [ap, mpre, mrec, _] = ElevenPointInterpolatedAP(rec, prec)

        r = {
            'class' : c,
            'precision' : prec,
            'recall' : rec,
            'AP' : ap,
            'interpolated precision' : mpre,
            'interpolated recall' : mrec,
            'total positives' : npos,
            'total TP' : np.sum(TP),
            'total FP' : np.sum(FP)
        }

        result.append(r)

    return result

In [29]:
boxA = ['15', '0.549740', '0.350926', '0.359896', '0.620370']
boxB = ['0' , '0.570312', '0.375', '0.36875' , '0.535185','0.89402']
boxA = [float(i) for i in boxA]
boxB = [float(i) for i in boxB]
print(boxA)
print(boxB)

[15.0, 0.54974, 0.350926, 0.359896, 0.62037]
[0.0, 0.570312, 0.375, 0.36875, 0.535185, 0.89402]


In [30]:
boxA = convertToAbsoluteValues(boxA)
boxB = convertToAbsoluteValues(boxB)
print(boxA)
print(boxB)

[0.369792, 0.04074100000000003, 0.729688, 0.661111]
[0.38593700000000003, 0.10740749999999999, 0.7546870000000001, 0.6425925]


In [31]:
IoU = iou(boxA, boxB)
print(IoU)

0.9201546131995996


In [32]:
import glob

In [39]:
labelPath = 'C:/Users/qorwl/yolov5_py38/yolov5/test_vid_split/label'
imagePath = 'C:/Users/qorwl/yolov5_py38/yolov5/runs/detect/test/label'

detections, groundtruths, classes = boundingBoxes( labelPath , imagePath )

NotADirectoryError: [WinError 267] 디렉터리 이름이 올바르지 않습니다: 'C:/Users/qorwl/yolov5_py38/yolov5/test_vid_split/label\\frame1.txt'